In [ ]:
# Усов Руслан Николаевич
# Тема преддипломной практики: «Разработка алгоритмов машинного обучения для обнаружения ошибок в текстах (на примере Частного образовательного учреждения высшего образования «Московский университет имени С.Ю. Витте»)»

# Построение модели

## Установка зависимостей

In [ ]:
## Установка зависимостей

print("Начало установки зависимостей...")
!pip install pandas scikit-learn tensorflow matplotlib numpy
print("Установка зависимостей завершена.")

## Импорты

In [1]:
print("Установка зависимостей и импорт библиотек - начало")
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib
import matplotlib.pyplot as plt
print("Зависимости установлены и библиотеки импортированы:")

print('matplotlib', matplotlib.__version__)
print('numpy', np.__version__)
print('tensorflow', tf.__version__)


Установка зависимостей и импорт библиотек - начало
Зависимости установлены и библиотеки импортированы:
matplotlib 3.10.3
numpy 2.1.3
tensorflow 2.19.0


## Создание единой конфигурации для модели

In [2]:
print("Конфигурация - начало")
config = {
    "max_vocab_size": 15000,
    "max_sequence_len": 25,
    "embedding_dim": 128,
    "lstm_units": 64,
    "dropout_rate": 0.3,
    "batch_size": 64,
    "epochs": 15,
    "patience": 3,
    "model_dir": "../model/model.keras",
    "best_model_path": "../model/b2/best_model.keras"
}
print(f"Конфиг установлен: {config}")

Конфигурация - начало
Конфиг установлен: {'max_vocab_size': 15000, 'max_sequence_len': 25, 'embedding_dim': 128, 'lstm_units': 64, 'dropout_rate': 0.3, 'batch_size': 64, 'epochs': 15, 'patience': 3, 'model_dir': '../model/model.keras', 'best_model_path': '../model/b2/best_model.keras'}


## Загрузка df, создание словаря

In [4]:
# Загрузка и подготовка данных
print("Загрузка датасета - начало")
df = pd.read_csv("../data/processed/dataset.csv")
print(f"Загружено {len(df)} записей")
# Приводим к строкам и убираем NaN
for col in ["noisy", "correct"]:
    df[col] = df[col].fillna("").astype(str)
# Создаем словарь исправлений
corrections = dict(zip(df["noisy"], df["correct"]))
# Готовим тексты и метки
texts = df["noisy"].tolist() + df["correct"].tolist()
labels = [1] * len(df) + [0] * len(df)
print("Данные загружены и очищены")

Загрузка датасета - начало
Загружено 39119 записей
Данные загружены и очищены


## Предобработка текста (char-level)

In [5]:
# Предобработка текста (char-level)
print("Предобработка - начало")
assert all(isinstance(t, str) for t in texts), "Некоторые тексты не являются строками!"
tokenizer = Tokenizer(char_level=True, num_words=config["max_vocab_size"], oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=config["max_sequence_len"], padding='post')
labels_array = np.array(labels)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(padded, labels_array, test_size=0.2, random_state=42)
print("Предобработка выолнена: char-level последовательности созданы")

Предобработка - начало
Предобработка выолнена: char-level последовательности созданы


In [6]:
# Сериализуем в JSON tokenizer и имортируем
print('Начало сериализации')

tokenizer_json = tokenizer.to_json()
with open("../model/tokenizer/tokenizer.json", "w", encoding="utf-8") as f:
    f.write(tokenizer_json)
    
print('Сериализация завершена')

Начало сериализации


FileNotFoundError: [Errno 2] No such file or directory: '../model/tokenizer/tokenizer.json'